In [ ]:
# Load LSTM network and generate text
import sys
import numpy
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint


# load csv into list file
lis=[]
with open('/Users/dzy/Desktop/变量/Extracurricular/Deep Learning Music Composition/MusicComposition/data/processed_csv_bin/83.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            a=row[0].split()
            lis.append([int(i) for i in a])


# summarize the loaded data
totalmidiclicks = len(lis)
print("Total midi clicks: ", totalmidiclicks)


# prepare the dataset of input to output pairs encoded as integers
seq_length = 64
dataX = []
dataY = []
for i in range(0, totalmidiclicks - seq_length, 1):
    seq_in = lis[i:i + seq_length]
    seq_out = lis[i + seq_length]
    dataX.append([singleclick for singleclick in seq_in])
    dataY.append(seq_out)
n_patterns = len(dataX)
print("Total iterations: ", n_patterns)


# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 128))
Y = numpy.reshape(dataY,(n_patterns,128))



# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


# fit the model
model.fit(X, Y, epochs=1000, batch_size=64, callbacks=callbacks_list)

In [ ]:
from keras.models import model_from_json
model.load_weights("/Users/dzy/Desktop/变量/Extracurricular/Deep Learning Music Composition/MusicComposition/weights-improvement-149-3.2727-bigger.hdf5", by_name=False)
model.summary()


In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")

music = pattern

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 128))
    prediction = model.predict(x, verbose=0)
    
    result = numpy.round(prediction*35)
    result = numpy.where(result>1,1,result)
    seq_in = [value for value in pattern]
    listresult = result.tolist()
    listresult = [[int(i) for i in listresult[0]]]
    seq_in.append(listresult[0])
    pattern = seq_in[1:]
    music.append(listresult[0])
print(music)



with open('music83_150epoch_64seqlength_1000length_*35.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for n in music:
        m = str(n)
        m = m.replace(']','\n')
        m = m.replace('[','')
        m = m.replace(']','')
        m = m.replace(',','')
        csvfile.write(m)



In [ ]:
                ### Below are the function oriented python code equivalent to above ###
                ### Below are the function oriented python code equivalent to above ###
                ### Below are the function oriented python code equivalent to above ###

In [ ]:
# Load LSTM network and generate text
import sys
import numpy
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint

def TrainYourModel(file,seq_length,epoch,batch_sizes):
    """
    file: string variable representing the file path for our processed_csv file
    seq_length: int variable the strength of short term memory; 
                a bigger value yields better short term memory, 
                but needs more time to be trained.
    epoch: int variable representing iteration of training
    batch_sizes: int variable representing how many how many file to be trained at one iteration
    """
    # load csv into list file
    lis=[]
    with open(file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                a=row[0].split()
                lis.append([int(i) for i in a])


    # summarize the loaded data
    totalmidiclicks = len(lis)
    print("Total midi clicks: ", totalmidiclicks)


    # prepare the dataset of input to output pairs encoded as integers
    dataX = []
    dataY = []
    for i in range(0, totalmidiclicks - seq_length, 1):
        seq_in = lis[i:i + seq_length]
        seq_out = lis[i + seq_length]
        dataX.append([singleclick for singleclick in seq_in])
        dataY.append(seq_out)
    n_patterns = len(dataX)
    print("Total iterations: ", n_patterns)


    # reshape X to be [samples, time steps, features]
    X = numpy.reshape(dataX, (n_patterns, seq_length, 128))
    Y = numpy.reshape(dataY,(n_patterns,128))



    # define the LSTM model
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(Y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')


    # define the checkpoint
    filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]


    # fit the model
    model.fit(X, Y, epochs=epoch, batch_size=batch_sizes, callbacks=callbacks_list)

In [ ]:
def LoadTheModel(Model):
"""
The variable "Model" should be a string variable representing the file path for .hdf5 file which is our model 
But if the file path is already been specified, Model can be just the name of the .hdf5 file
"""
    from keras.models import model_from_json
    model.load_weights(Model, by_name=False)
    model.summary()


In [ ]:
def GenerateMusic(music_length,output):
    """
    music_length: int value representing how many 0.1 seconds the music 
                  will be (generated music might be shorted than setted)
    output: string value representing the name of the generated music
    """
    # pick a random seed
    start = numpy.random.randint(0, len(dataX)-1)
    pattern = dataX[start]
    print("Seed:")

    music = pattern

    # generate characters
    for i in range(music_length):
        x = numpy.reshape(pattern, (1, len(pattern), 128))
        prediction = model.predict(x, verbose=0)

        result = numpy.round(prediction*35)
        result = numpy.where(result>1,1,result)
        seq_in = [value for value in pattern]
        listresult = result.tolist()
        listresult = [[int(i) for i in listresult[0]]]
        seq_in.append(listresult[0])
        pattern = seq_in[1:]
        music.append(listresult[0])
    print(music)



    with open(output, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for n in music:
            m = str(n)
            m = m.replace(']','\n')
            m = m.replace('[','')
            m = m.replace(']','')
            m = m.replace(',','')
            csvfile.write(m)
